In [1]:
import pandas as pd

In [2]:
from helper import load_mistral_api_key
api_key, dlai_endpoint = load_mistral_api_key(ret_key=True)

In [3]:
data = {
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}
df = pd.DataFrame(data)

In [4]:
df

,transaction_id,customer_id,payment_amount,payment_date,payment_status
0,T1001,C001,125.50,2021-10-05,Paid
1,T1002,C002,89.99,2021-10-06,Unpaid
2,T1003,C003,120.00,2021-10-07,Paid
3,T1004,C002,54.30,2021-10-05,Paid
4,T1005,C001,210.20,2021-10-08,Pending


In [5]:
data = """
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}
"""
transaction_id = "T1001"

prompt = f"""
Given the following data, what is the payment status for \
 transaction_id={transaction_id}?

data:
{data}

"""

In [6]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage


def mistral(user_message, model="mistral-small-latest", is_json=False):
    client = MistralClient(api_key=api_key)
    messages = [ChatMessage(role="user", content=user_message)]

    if is_json:
        chat_response = client.chat(
            model=model, messages=messages, response_format={"type": "json_object"}
        )
    else:
        chat_response = client.chat(model=model, messages=messages)

    return chat_response.choices[0].message.content

In [7]:
response = mistral(prompt)
print(response)

The payment status for transaction_id=T1001 is "Paid".


#### Step 1. User: specify tools and query

In [8]:
import json

def retrieve_payment_status(df, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"status": df[df.transaction_id == transaction_id].payment_status.item()}
        )
    return json.dumps({"error": "transaction id not found."})

In [9]:
status = retrieve_payment_status(df, transaction_id="T1001")
print(status)

{"status": "Paid"}


In [10]:
def retrieve_payment_date(df, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"date": df[df.transaction_id == transaction_id].payment_date.item()}
        )
    return json.dumps({"error": "transaction id not found."})

In [11]:
date = retrieve_payment_date(df, transaction_id="T1002")
print(date)

{"date": "2021-10-06"}


In [12]:
tool_payment_status = {
    "type": "function",
    "function": {
        "name": "retrieve_payment_status",
        "description": "Get payment status of a transaction",
        "parameters": {
            "type": "object",
            "properties": {
                "transaction_id": {
                    "type": "string",
                    "description": "The transaction id.",
                }
            },
            "required": ["transaction_id"],
        },
    },
}

In [13]:
tool_payment_date = {
    "type": "function",
    "function": {
        "name": "retrieve_payment_date",
        "description": "Get payment date of a transaction",
        "parameters": {
            "type": "object",
            "properties": {
                "transaction_id": {
                    "type": "string",
                    "description": "The transaction id.",
                }
            },
            "required": ["transaction_id"],
        },
    },
}

In [14]:
tools = [tool_payment_status, tool_payment_date]

#### functools

In [15]:
import functools

In [16]:
names_to_functions = {
    "retrieve_payment_status": functools.partial(retrieve_payment_status, df=df),
    "retrieve_payment_date": functools.partial(retrieve_payment_date, df=df),
}

In [17]:
names_to_functions["retrieve_payment_status"](transaction_id="T1001")

'{"status": "Paid"}'

#### query

In [18]:
from mistralai.models.chat_completion import ChatMessage

chat_history = [
    ChatMessage(role="user", content="What's the status of my transaction?")
]

#### Step 2. Model: Generate function arguments

In [19]:
from mistralai.client import MistralClient

model = "mistral-large-latest"

client = MistralClient(api_key=api_key)

response = client.chat(
    model=model, messages=chat_history, tools=tools, tool_choice="auto"
)



In [20]:
print(response)

id='1768d17947b541b192259864eb8b9a5b' object='chat.completion' created=1718647797 model='mistral-large-latest' choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='To get the status of your transaction, I need the transaction ID. Could you please provide it?', name=None, tool_calls=None), finish_reason=<FinishReason.stop: 'stop'>)] usage=UsageInfo(prompt_tokens=161, total_tokens=181, completion_tokens=20)


In [21]:
response.choices[0].message.content

'To get the status of your transaction, I need the transaction ID. Could you please provide it?'

#### Save the chat history

In [22]:
chat_history.append(
    ChatMessage(role="assistant", content=response.choices[0].message.content)
)
chat_history.append(ChatMessage(role="user", content="My transaction ID is T1001."))
chat_history

[ChatMessage(role='user', content="What's the status of my transaction?", name=None, tool_calls=None),
 ChatMessage(role='assistant', content='To get the status of your transaction, I need the transaction ID. Could you please provide it?', name=None, tool_calls=None),
 ChatMessage(role='user', content='My transaction ID is T1001.', name=None, tool_calls=None)]

In [23]:
response = client.chat(
    model=model, messages=chat_history, tools=tools, tool_choice="auto"
)

In [24]:
response.choices[0].message

ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='1T7YPzNgg', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'))])

In [25]:
chat_history.append(response.choices[0].message)

#### Step 3. User: Execute function to obtain tool results

In [26]:
tool_function = response.choices[0].message.tool_calls[0].function
print(tool_function)

name='retrieve_payment_status' arguments='{"transaction_id": "T1001"}'


In [27]:
tool_function.name

'retrieve_payment_status'

In [28]:
tool_function.arguments

'{"transaction_id": "T1001"}'

In [29]:
args = json.loads(tool_function.arguments)
print(args)

{'transaction_id': 'T1001'}


In [30]:
function_result = names_to_functions[tool_function.name](**args)
function_result

'{"status": "Paid"}'

In [31]:
tool_msg = ChatMessage(role="tool", name=tool_function.name, content=function_result)
chat_history.append(tool_msg)

In [32]:
chat_history

[ChatMessage(role='user', content="What's the status of my transaction?", name=None, tool_calls=None),
 ChatMessage(role='assistant', content='To get the status of your transaction, I need the transaction ID. Could you please provide it?', name=None, tool_calls=None),
 ChatMessage(role='user', content='My transaction ID is T1001.', name=None, tool_calls=None),
 ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='1T7YPzNgg', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'))]),
 ChatMessage(role='tool', content='{"status": "Paid"}', name='retrieve_payment_status', tool_calls=None)]

#### Step 4. Model: Generate final answer

In [33]:
response = client.chat(model=model, messages=chat_history)
print(response.choices[0].message.content)

According to the information I've retrieved, your transaction with ID T1001 has been successfully paid.


### Try it for yourself!
#### Try asking another question about the data, such as "how much did I pay my recent order?
#### You can be customer "T1002", for instance

In [34]:
chat_history = []

In [35]:
chat_history = [
    ChatMessage(role="user", content="How much did I pay my recent order? my transaction id is T1002")
]

In [36]:
client = MistralClient(api_key=api_key)

response = client.chat(
    model=model, messages=chat_history, tools=tools, tool_choice="auto"
)


In [37]:
print(response.choices[0].message)

role='assistant' content="I'm sorry for any confusion, but I'm unable to provide the exact amount you paid for your recent order as I don't have a function that retrieves the payment amount. However, I can help you check the status or the date of your payment. Would you like to know either of those?" name=None tool_calls=None


In [38]:
chat_history.append(ChatMessage(role=response.choices[0].message.role, content=response.choices[0].message.content))

In [39]:
chat_history.append(ChatMessage(role='user', content='Okay then, what is the date of my last paiment?'))

In [40]:
chat_history

[ChatMessage(role='user', content='How much did I pay my recent order? my transaction id is T1002', name=None, tool_calls=None),
 ChatMessage(role='assistant', content="I'm sorry for any confusion, but I'm unable to provide the exact amount you paid for your recent order as I don't have a function that retrieves the payment amount. However, I can help you check the status or the date of your payment. Would you like to know either of those?", name=None, tool_calls=None),
 ChatMessage(role='user', content='Okay then, what is the date of my last paiment?', name=None, tool_calls=None)]

In [41]:
response = client.chat(model=model, messages=chat_history, tools=tools, tool_choice='auto')

In [42]:
response

ChatCompletionResponse(id='6278a81924f241ba8696eb039f0bb14d', object='chat.completion', created=1718647877, model='mistral-large-latest', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='NdFqUUcnI', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_date', arguments='{"transaction_id": "T1002"}'))]), finish_reason=<FinishReason.tool_calls: 'tool_calls'>)], usage=UsageInfo(prompt_tokens=251, total_tokens=281, completion_tokens=30))

In [43]:
chat_history.append(response.choices[0].message)

In [44]:
chat_history

[ChatMessage(role='user', content='How much did I pay my recent order? my transaction id is T1002', name=None, tool_calls=None),
 ChatMessage(role='assistant', content="I'm sorry for any confusion, but I'm unable to provide the exact amount you paid for your recent order as I don't have a function that retrieves the payment amount. However, I can help you check the status or the date of your payment. Would you like to know either of those?", name=None, tool_calls=None),
 ChatMessage(role='user', content='Okay then, what is the date of my last paiment?', name=None, tool_calls=None),
 ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='NdFqUUcnI', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_date', arguments='{"transaction_id": "T1002"}'))])]

In [45]:
called_function = response.choices[0].message.tool_calls[0].function

In [46]:
called_function.name

'retrieve_payment_date'

In [47]:
called_function.arguments

'{"transaction_id": "T1002"}'

The function arguments are expected to be in a Python dictionary and not a string.
To make this string into a dictionary, you can use json.loads()

In [48]:
args = json.loads(called_function.arguments)

In [49]:
function_res = names_to_functions[called_function.name](**args)
function_res

'{"date": "2021-10-06"}'

In [50]:
tool_msg = ChatMessage(role='tool', name=called_function.name, content=function_res)
chat_history.append(tool_msg)

In [51]:
chat_history

[ChatMessage(role='user', content='How much did I pay my recent order? my transaction id is T1002', name=None, tool_calls=None),
 ChatMessage(role='assistant', content="I'm sorry for any confusion, but I'm unable to provide the exact amount you paid for your recent order as I don't have a function that retrieves the payment amount. However, I can help you check the status or the date of your payment. Would you like to know either of those?", name=None, tool_calls=None),
 ChatMessage(role='user', content='Okay then, what is the date of my last paiment?', name=None, tool_calls=None),
 ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='NdFqUUcnI', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_date', arguments='{"transaction_id": "T1002"}'))]),
 ChatMessage(role='tool', content='{"date": "2021-10-06"}', name='retrieve_payment_date', tool_calls=None)]

In [52]:
response = client.chat(model=model, messages=chat_history)


In [54]:
print(response.choices[0].message.content)

Based on the information I've received, the date of your last payment with transaction ID T1002 was October 6, 2021.


### Parallel Function Calling

In [55]:
response = client.chat(
    model=model,
    messages=[
        ChatMessage(
            role='user',
            content='What is the date and status of my Transaction ID T1005?')
            ],
    tools=tools, 
    tool_choice='auto' 

    )

In [58]:
print(response.choices[0])

index=0 message=ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='c7Qt4yByC', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1005"}')), ToolCall(id='J80C6m9xu', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_date', arguments='{"transaction_id": "T1005"}'))]) finish_reason=<FinishReason.tool_calls: 'tool_calls'>
